# Access Azure resources from an online endpoint with a user-assigned managed identity

Additional packages are required for this example including 
* azure.mgmt.msi
* azure.mgmt.storage

Install them with the following code:

In [ ]:
!pip install azure-mgmt-msi
!pip install azure-mgmt-storage

## 1. Configure variables

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"

endpoint_name = "<ENDPOINT_NAME>"

storage_account_name = "<STORAGE_ACCOUNT_NAME>"
storage_container_name = "<CONTAINER_TO_ACCESS>"
file_name = "<FILE_TO_ACCESS>"

uai_name = "<USER_ASSIGNED_IDENTITY_NAME>"

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, AzureCliCredential
from azure.ai.ml.entities import (
    ManagedOnlineDeployment,
    ManagedOnlineEndpoint,
    Model,
    CodeConfiguration,
    Environment,
)

In [ ]:
credential = AzureCliCredential()
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

workspace_location = ml_client.workspaces.get(workspace_name).location

## 2. Configure deployment

In [ ]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name, auth_mode="key")

In [ ]:
deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=Model(path="../../model-1/model/"),
    code_configuration=CodeConfiguration(
        code="../../model-1/onlinescoring/", scoring_script="score_managedidentity.py"
    ),
    environment=Environment(
        conda_file="../../model-1/environment/conda.yml",
        image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1",
    ),
    instance_type="Standard_DS2_v2",
    instance_count=1,
    environment_variables={
        "STORAGE_ACCOUNT_NAME": storage_account_name,
        "STORAGE_CONTAINER_NAME": storage_container_name,
        "FILE_NAME": file_name,
        # We will update this after creating an identity
        "UAI_CLIENT_ID": "uai_client_id_place_holder",
    },
)

## 3. Create the managed identity

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.msi import ManagedServiceIdentityClient
from azure.mgmt.msi.models import Identity

In [ ]:
credential = DefaultAzureCredential()
msi_client = ManagedServiceIdentityClient(
    subscription_id=subscription_id,
    credential=credential,
)

In [ ]:
msi_client.user_assigned_identities.create_or_update(
    resource_group_name=resource_group,
    resource_name=uai_name,
    parameters=Identity(location=workspace_location),
)

In [ ]:
uai_identity = msi_client.user_assigned_identities.get(
    resource_group_name=resource_group,
    resource_name=uai_name,
)

## 4. Create storage account and container

In [ ]:
from azure.identity import DefaultAzureCredential, AzureCliCredential
from azure.mgmt.storage import StorageManagementClient
from azure.storage.blob import ContainerClient
from azure.mgmt.storage.models import Sku, StorageAccountCreateParameters, BlobContainer

In [ ]:
credential = AzureCliCredential()
storage_client = StorageManagementClient(
    credential=credential, subscription_id=subscription_id
)

In [ ]:
storage_account_parameters = StorageAccountCreateParameters(
    sku=Sku(name="Standard_LRS"), kind="Storage", location=workspace_location
)

poller = storage_client.storage_accounts.begin_create(
    resource_group_name=resource_group,
    account_name=storage_account_name,
    parameters=storage_account_parameters,
)

poller.wait()

storage_account = poller.result()

In [ ]:
blob_container = storage_client.blob_containers.create(
    resource_group_name=resource_group,
    account_name=storage_account_name,
    container_name=storage_container_name,
    blob_container=BlobContainer(),
)

In [ ]:
res = storage_client.storage_accounts.list_keys(
    resource_group_name=resource_group,
    account_name=storage_account_name,
)
key = res.keys[0].value

In [ ]:
container_client = ContainerClient(
    account_url=storage_account.primary_endpoints.blob,
    container_name=storage_container_name,
    credential=key,
)

In [ ]:
file_path = "hello.txt"
with open(file_path, "rb") as f:
    container_client.upload_blob(name=file_name, data=f.read())

## 5. Create an online endpoint

In [ ]:
# Create the endpoint
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint)

In [ ]:
# Update its identity
endpoint.identity = endpoint.identity.from_dict(
    {"type": "UserAssigned", "user_assigned_identities": {uai_identity.id: {}}}
)

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint)

## 6. Give access permission to the managed identity

In [ ]:
from azure.mgmt.authorization import AuthorizationManagementClient
from azure.mgmt.authorization.models import (
    RoleAssignment,
    RoleDefinition,
    RoleAssignmentCreateParameters,
    RoleAssignmentProperties,
    RoleAssignmentPropertiesWithScope,
)
import uuid

auth_client = AuthorizationManagementClient(
    credential=credential, subscription_id=subscription_id
)

In [ ]:
uai_identity = msi_client.user_assigned_identities.get(
    resource_group_name=resource_group, resource_name=uai_name
)
uai_principal_id = uai_identity.principal_id
uai_client_id = uai_identity.client_id

In [ ]:
workspace = ml_client.workspaces.get(workspace_name)
container_registry = workspace.container_registry

In [ ]:
role_name = "Storage Blob Data Reader"
scope = storage_account.id

role_defs = auth_client.role_definitions.list(scope=scope)
role_def = next((r for r in role_defs if r.role_name == role_name))

auth_client.role_assignments.create(
    scope=scope,
    role_assignment_name=uuid.uuid4(),
    parameters=RoleAssignmentProperties(
        role_definition_id=role_def.id, principal_id=uai_principal_id
    ),
)

In [ ]:
role_name = "AcrPull"
scope = container_registry

role_defs = auth_client.role_definitions.list(scope=scope)
role_def = next((r for r in role_defs if r.role_name == role_name))

auth_client.role_assignments.create(
    scope=scope,
    role_assignment_name=uuid.uuid4(),
    parameters=RoleAssignmentProperties(
        role_definition_id=role_def.id, principal_id=uai_principal_id
    ),
)

## 7. Create a deployment with your configuration

In [ ]:
deployment.uai_client_id = uai_client_id

deployment = ml_client.online_deployments.begin_create_or_update(deployment)

## 8. Confirm your endpoint deployed successfully

In [ ]:
sample_data = "../../model-1/sample-request.json"
ml_client.online_endpoints.invoke(endpoint_name=endpoint_name, request_file=sample_data)

## 9. Delete the endpoint and storage account

In [ ]:
ml_client.online_endpoints.begin_delete(endpoint_name)

In [ ]:
storage_client.storage_accounts.delete(
    resource_group_name=resource_group, account_name=storage_account_name
)